# Neural Network to regconise circles, squares, triangles.

# Imports

In [1]:
import numpy as np   # linear algebra 
import pandas as pd  # data processing

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
import os
os.chdir('../input/') # Changing the directory

In [3]:
import pandas as pd
df=pd.DataFrame()
path=[]
label=[]
code=[]
lst=["circles","squares","triangles"]
for i in lst:
  super=os.listdir('basicshapes/shapes/'+i)
  for j in super:
    path.append(j)
    label.append(i)
    if i=="circles":
      code.append(0)
    elif i=="squares":
      code.append(1)
    else:
      code.append(2)
df["label"]=label
df["path"]=path
df["code"]=code

import cv2
from tqdm import tqdm
import numpy as np
import random
import tensorflow as tf

image=[]
lst=["circles","squares","triangles"]
for i in tqdm(lst):
  b='basicshapes/shapes/'+i
  a=os.listdir('basicshapes/shapes/'+i)
  for j in a:
    image.append(cv2.imread(b+'/'+j))
df["image"]=image

for index,row in df.iterrows():
  print(np.shape(row.image))

dom=[]
for i in range(0,240):
  dom.append(0)
for i in range(240,270):
  dom.append(1)
for i in range(270,300):
  dom.append(2)
random.shuffle(dom)
df["subset"]=dom

df

df.groupby("label")["subset"].value_counts()

lst=["circles","squares","triangles"]
new_df=pd.DataFrame()
for i in lst:
  new_df=df[df.label==i].sample(10)
  for j in new_df["image"]:
    print(j)

def generate(df,batch):
  num=batch/3
  while 1:
    image=[]
    label=[]
    for i in lst:
      new_df=df[df.label==i].sample(batch//3,replace=True)
      for j in new_df["image"]:
        image.append(j)
      for j in new_df["code"]:
        label.append(j)
    image=np.array(image)
    label=np.array(label)
    label=tf.keras.utils.to_categorical(label)
    yield image,label


x,y=next(generate(df,30))
print(x.shape)

print(len(df[(df.subset==1) & (df.label=="triangles")]))

model = tf.keras.Sequential(
    [

        tf.keras.Input(shape=(28,28,3)),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(3, activation="softmax"),
    ]
)

model.summary()

batch_size = 30
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(generate(df[df.subset==0],batch_size), batch_size=batch_size, epochs=epochs,steps_per_epoch=100,validation_steps=100,validation_data=generate(df[df.subset==1],batch_size))

score = model.evaluate(generate(df[df.subset==2],batch_size), verbose=0,steps=100)
print("Test loss:", score[0])
print("Test accuracy:", score[1])